In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [209]:
dataset = pd.read_csv(r'Corona_NLP_train.csv',encoding ='latin-1')
x = dataset.iloc[:,4:-1].values
y = dataset.iloc[:,-1].values
X=x.flatten()

In [210]:
for i in range(len(y)):
    if y[i]=="Positive":
        y[i]=0
    elif y[i]=="Negative":
        y[i]=1
    elif y[i]=="Extremely Positive":
        y[i]=2
    elif y[i]=="Extremely Negative":
        y[i]=3
    elif y[i]=="Neutral":
        y[i]=4   
    

In [211]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1000)

In [212]:
from sklearn.feature_extraction.text import CountVectorizer
review_vectorizer = CountVectorizer()
review_vectorizer.fit(X_train)
Xlr_train = review_vectorizer.transform(X_train)
Xlr_test  = review_vectorizer.transform(X_test)
Xlr_train 

<30867x65786 sparse matrix of type '<class 'numpy.int64'>'
	with 863156 stored elements in Compressed Sparse Row format>

In [213]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
Xann_train = tokenizer.texts_to_sequences(X_train)
Xann_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1  
print(X_train[0])
print(Xann_train[0]) 

#coronavirus and ?decline in oil prices = global recession.
[11, 5, 1039, 10, 94, 21, 213, 836]


In [214]:
from keras_preprocessing.sequence import pad_sequences
maxlen = 100
Xann_train = pad_sequences(Xann_train, padding='post', maxlen=maxlen)
Xann_test = pad_sequences(Xann_test, padding='post', maxlen=maxlen)
print(Xann_train[0, :]) 

[  11    5 1039   10   94   21  213  836    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [215]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Xann_train = sc.fit_transform(Xann_train)
Xann_test = sc.transform(Xann_test)

In [216]:
x_tensor = tf.convert_to_tensor(Xann_train, dtype=tf.float32)
y_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

# RNN

In [246]:
x_tensor_re = np.reshape(x_tensor, (x_tensor.shape[0], -1, x_tensor.shape[1]))

In [247]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [248]:
regressor = Sequential()

In [249]:
regressor.add(LSTM(input_dim = vocab_size,units = 50, return_sequences = True, input_shape = (30867 , 100)))
regressor.add(Dropout(0.2))

In [250]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [251]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [252]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [253]:
regressor.add(Dense(units = 5))

In [254]:
regressor.compile(optimizer = 'adam', loss = "huber_loss")

In [255]:
regressor.fit(x_tensor_re, y_tensor, epochs = 3,validation_split=0.12, batch_size = 32)

Epoch 1/3
849/849 [==============================] - 34s 14ms/step - loss: 0.8723 - val_loss: 0.8342
Epoch 2/3
849/849 [==============================] - 9s 11ms/step - loss: 0.8319 - val_loss: 0.8340
Epoch 3/3
849/849 [==============================] - 8s 9ms/step - loss: 0.8279 - val_loss: 0.8359


# ANN

In [171]:
ann = tf.keras.models.Sequential()

In [172]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [173]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [174]:
ann.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

In [177]:
ann.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])

In [178]:
ann.fit(x_tensor, y_tensor, batch_size = 32, epochs = 20)

Epoch 1/20
965/965 [==============================] - 3s 2ms/step - loss: 0.2972 - accuracy: 0.1889
Epoch 2/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2780 - accuracy: 0.2247
Epoch 3/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2779 - accuracy: 0.2237
Epoch 4/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2271
Epoch 5/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2261
Epoch 6/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2252
Epoch 7/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2235
Epoch 8/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2222
Epoch 9/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2197
Epoch 10/20
965/965 [==============================] - 2s 2ms/step - loss: 0.2778 - accuracy: 0.2185

# 

# CNN

In [217]:
from keras.models import Sequential
from keras import layers 

embedding_dim =200

In [218]:
cnnmodel = Sequential()

In [219]:
cnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

In [220]:
cnnmodel.add(layers.Conv1D(128, 5, activation='relu'))

In [221]:
cnnmodel.add(layers.GlobalMaxPooling1D())

In [222]:
cnnmodel.add(layers.Dense(10, activation='relu'))

In [223]:
cnnmodel.add(layers.Dense(5, activation='sigmoid'))

In [224]:
cnnmodel.compile(optimizer='adam',
               loss='kullback_leibler_divergence',
               metrics=['accuracy'])
cnnmodel.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 200)          13926800  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 96, 128)           128128    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
Total params: 14,056,273
Trainable params: 14,056,273
Non-trainable params: 0
_________________________________________________________________


In [225]:
cnnmodel.fit(x_tensor, y_tensor,
                     epochs=1,
                     validation_split=0.12,
                     batch_size=128)

213/213 [==============================] - 55s 252ms/step - loss: 0.1836 - accuracy: 0.1617 - val_loss: 7.3428e-04 - val_accuracy: 0.1557


# Predicting on a sample Text

# Processing Text

In [257]:
test_text = ['Happy happy happy happy happy']

In [258]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(test_text)
x_test1 = tokenizer.texts_to_sequences(test_text)
vocab_size = len(tokenizer.word_index) + 1  
print(test_text)
print(x_test1) 

['Happy happy happy happy happy']
[[1, 1, 1, 1, 1]]


In [259]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100
x_test1 = pad_sequences(x_test1, padding='post', maxlen=maxlen)
print(x_test1[0, :]) 

[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# ANN Predict

In [260]:
y_pred = ann.predict(Xann_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),-1), y_test.reshape(len(y_test),1)),1))

[[True True True True True 3]
 [True True True True True 0]
 [True True True True True 1]
 ...
 [True True True True True 4]
 [True True True True True 0]
 [True True True True True 0]]


In [261]:
print(y_test)

[3 0 1 ... 4 0 0]


In [262]:
print(ann.predict(x_test1))

[[0.9996816 0.9999244 0.9996966 0.9998295 0.9998667]]


# CNN Predict

In [263]:
result = cnnmodel.predict(x_test1)
print(result)

[[0.9994276  0.9997387  0.99999225 0.9999621  0.9996623 ]]


# RNN Predict

In [268]:
x_test1.reshape(x_test1.shape[0] ,-1 ,1)

array([[[1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],

In [269]:
print(regressor.predict(sc.transform(x_test1)>0.5))

ValueError: in user code:

    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\himan\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:1569 run_step  **
  